# Проект для «Викишоп»

Интернет-магазин запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Задача: обучить модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Цель: создать модель со значением метрики качества *F1* не меньше 0.75. 



Для выполнения данной задачи совершим сделующие действия:

1. Загрузим библиотеки и посмотрим, что у нас есть
2. Лемматизируем и очистим текст
3. Обучим модель
4. Проверим модель на тестовой выборке

## Подготовка

импортируем библиотеки и посмотрим на данные

In [ ]:
import pandas as pd
import nltk
from nltk.stem import WordNetLemmatizer 
from nltk.corpus import wordnet
from collections import defaultdict
from nltk.tokenize import word_tokenize
from nltk import pos_tag
import re
from tqdm import tqdm
from sklearn.feature_extraction.text import CountVectorizer 
from nltk.corpus import stopwords 
#nltk.download('stopwords') 
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.metrics import f1_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier

In [ ]:
data = pd.read_csv('/datasets/toxic_comments.csv')

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159571 non-null  object
 1   toxic   159571 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


In [ ]:
text = data['text'].values

Лемматизируем и очистим текст

In [ ]:
tag_map = defaultdict(lambda : wordnet.NOUN)
tag_map['J'] = wordnet.ADJ
tag_map['V'] = wordnet.VERB
tag_map['R'] = wordnet.ADV
 
def lemmatize(text):
    tokens = word_tokenize(text)
    lmtzr = WordNetLemmatizer()
    lemma = []
    for token, tag in pos_tag(tokens):
        lemma.append(lmtzr.lemmatize(token, tag_map[tag[0]]))
   
    lemm_text = " ".join((re.sub(r'[^a-zA-Z ]', ' ', " ".join(lemma))).split())
     
    return lemm_text
 
nltk.download('averaged_perceptron_tagger')

tqdm.pandas()
data['lemm_text'] = (data['text']).apply(lemmatize)

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


## Обучение

Обучим несколько моделей. С текстом хорошо работают LogisticRegression, MultinomialNB и SGD Classifier. Посмотрим, кто справится лучше

In [ ]:
corpus = list(data['lemm_text'])

#corpus_train, corpus_test = train_test_split(corpus, test_size=0.2, train_size=0.8, random_state=12345)
count_vect = CountVectorizer(ngram_range=(2, 2)) 
n_gramm = count_vect.fit_transform(corpus)
stop_words = set(stopwords.words('english'))

target = data['toxic']
features = corpus
features_train, features_test, target_train, target_test = train_test_split(
    features, target, test_size=0.2, random_state=12345)

#corpus_train = corpus
#corpus_test = corpus
#target_train = data['toxic']
#target_test = data['toxic']

count_tf_idf = TfidfVectorizer(stop_words=stop_words)
tf_idf = count_tf_idf.fit_transform(corpus) 
tf_idf_train = count_tf_idf.fit_transform(features_train)  
tf_idf_test = count_tf_idf.transform(features_test) 


In [ ]:
nb = MultinomialNB()
nb.fit(tf_idf_train, target_train)
predictions = nb.predict(tf_idf_test)
print(f1_score(predictions, target_test))

0.351715374841169


In [ ]:
model = LogisticRegression(random_state=12345, solver='liblinear', penalty='l1', tol=0.01)
model.fit(tf_idf_train, target_train)
predictions = model.predict(tf_idf_test)
print(f1_score(predictions, target_test))

0.7717523975588493


In [ ]:

model = SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, random_state=42, max_iter=5, tol=None)
model.fit(tf_idf_train, target_train)
predictions = model.predict(tf_idf_test)
print(f1_score(predictions, target_test))

0.2711502535361623


## Выводы

Logistis Regression показала лучший результат, поэтому тестовую выборку проверим на ней

Вывод:
1) были изучены данные

2) лемматизирован и очищен текст

3) обучены модели

4) на тестовой выборке проверена модель, показаывшая лучшие результаты на обучении

5) получено значение f1 более 0,75